<a href="https://colab.research.google.com/github/yousenwang/langchain_llm/blob/main/convert_and_quantize_chinese_llama2_and_alpaca2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 转换并量化中文LLaMA-2和Alpaca-2模型

项目地址：https://github.com/ymcui/Chinese-LLaMA-Alpaca-2

⚠️ 内存消耗提示（确保刷出来的机器RAM大于以下要求）：
- 7B模型：15G+
- 13B模型：18G+
- 33B模型：22G+

💡 提示和小窍门：
- 免费用户默认的内存只有12G左右，不足以转换模型。**实测选择TPU的话有机会随机出35G内存**，建议多试几次
- Pro(+)用户请选择 “代码执行程序” -> “更改运行时类型” -> “高RAM”
- 程序莫名崩掉或断开连接就说明内存爆了
- 如果选了“高RAM”之后内存还是不够大的话，选择以下操作，有的时候会分配出很高内存的机器，祝你好运😄！
    - 可以把GPU或者TPU也选上（虽然不会用到）
    - 选GPU时，Pro(+)用户可选“A100”类型GPU

*温馨提示：用完之后注意断开运行时，选择满足要求的最低配置即可，避免不必要的计算单元消耗（Pro只给100个计算单元）。*

## 安装相关依赖

In [ ]:
!pip install git+https://github.com/huggingface/peft.git@13e53fc
!pip install transformers==4.31.0
!pip install sentencepiece==0.1.97
!pip install bitsandbytes==0.39.1

  Cloning https://github.com/huggingface/peft.git (to revision 13e53fc) to /tmp/pip-req-build-rth5yg2a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-rth5yg2a
  Running command git checkout -q 13e53fc
  Resolved https://github.com/huggingface/peft.git to commit 13e53fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=40651 sha256=a877759f5558de91ed69f0a01aae40d8c3d3098855b

## 克隆目录和代码

In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 2127, done.
remote: Counting objects: 100% (816/816), done.
remote: Compressing objects: 100% (323/323), done.
remote: Total 2127 (delta 553), reused 650 (delta 493), pack-reused 1311
Receiving objects: 100% (2127/2127), 22.03 MiB | 14.36 MiB/s, done.
Resolving deltas: 100% (1330/1330), done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 5703, done.
remote: Counting objects: 100% (2361/2361), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 5703 (delta 2235), reused 2051 (delta 2021), pack-reused 3342
Receiving objects: 100% (5703/5703), 4.43 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (3938/3938), done.


## 合并模型（以LLaMA-2-7B为例）

合并LoRA，生成全量模型权重。可以直接指定🤗模型库的地址，也可以是本地存放地址。
- 基模型：`meta-llama/Llama-2-7b-hf`（注意需要官方授权）
    - 这里使用一个平替（SHA256一致）做演示：`daryl149/llama-2-7b-hf`
- LoRA模型：`ziqingyang/chinese-llama-2-lora-7b`
- 输出格式：可选pth或者huggingface，这里选择huggingface

转换好的模型存放在`llama-2-7b-combined`目录。
如果你不需要量化模型，那么到这一步就结束了，可自行下载或者转存到Google Drive。

In [ ]:
!python ./Chinese-LLaMA-Alpaca-2/scripts/merge_llama2_with_chinese_lora_low_mem.py \
    --base_model daryl149/llama-2-7b-hf \
    --lora_model ziqingyang/chinese-llama-2-lora-7b \
    --output_type huggingface \
    --output_dir llama-2-7b-combined


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...
2023-07-27 07:44:07.692012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could n

## 检查修改config.json（如果使用meta原版则可跳过）

- Llama-2的config中途有一次更新。因为本教程里使用的是第三方的权重，并没有及时更新对应的`config.json`文件。

- 请手动打开`llama-2-7b-combined`文件夹下的config.json（可直接双击打开），将`max_position_embeddings`字段由`2048`改为`4096`。cmd/ctrl+s保存即可。

## 量化模型
接下来我们使用[llama.cpp](https://github.com/ggerganov/llama.cpp)工具对上一步生成的全量版本权重进行转换，生成量化模型。

### 编译工具

首先对llama.cpp工具进行编译。

In [ ]:
!cd llama.cpp && make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I LDFLAGS:  
I CC:       cc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
I CXX:      g++ (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0

cc  -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-m

### 模型转换为GGML格式（FP16）

这一步，我们将模型转换为GGML格式（FP16）。

In [ ]:
!cd llama.cpp && python convert.py ../llama-2-7b-combined

Loading model file ../llama-2-7b-combined/pytorch_model-00001-of-00002.bin
Loading model file ../llama-2-7b-combined/pytorch_model-00001-of-00002.bin
Loading model file ../llama-2-7b-combined/pytorch_model-00002-of-00002.bin
vocabtype: spm
Loading vocab file ../llama-2-7b-combined/tokenizer.model
params: n_vocab:55296 n_embd:4096 n_mult:256 n_head:32 n_layer:32
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  55296 x   4096  | type UnquantizedDataType(name='F16')
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  55296 x   4096  | type UnquantizedDataType(name='F16')
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type UnquantizedDataType(name='F16')
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type UnquantizedDataT

### 将FP16模型进行量化

我们进一步将FP16模型转换为量化模型，此处选择的是新版Q6_K方法，其效果非常接近FP16。

In [ ]:
!cd llama.cpp && ./quantize ../llama-2-7b-combined/ggml-model-f16.bin ../llama-2-7b-combined/ggml-model-q6_K.bin q6_K

main: build = 916 (b5472ea)
main: quantizing '../llama-2-7b-combined/ggml-model-f16.bin' to '../llama-2-7b-combined/ggml-model-q6_K.bin' as Q6_K
llama.cpp: loading model from ../llama-2-7b-combined/ggml-model-f16.bin
llama.cpp: saving model to ../llama-2-7b-combined/ggml-model-q6_K.bin
[   1/ 291]                tok_embeddings.weight -     4096 x 55296, type =    f16, quantizing to q6_K .. size =   432.00 MB ->   177.19 MB | hist: 
[   2/ 291]                          norm.weight -             4096, type =    f32, size =    0.016 MB
[   3/ 291]                        output.weight -     4096 x 55296, type =    f16, quantizing to q6_K .. size =   432.00 MB ->   177.19 MB | hist: 
[   4/ 291]         layers.0.attention.wq.weight -     4096 x  4096, type =    f16, quantizing to q6_K .. size =    32.00 MB ->    13.12 MB | hist: 
[   5/ 291]         layers.0.attention.wk.weight -     4096 x  4096, type =    f16, quantizing to q6_K .. size =    32.00 MB ->    13.12 MB | hist: 
[   6/ 291]   

### （可选）测试量化模型解码
至此已完成了所有转换步骤。
我们运行一条命令测试一下是否能够正常加载并进行输出。

In [ ]:
!cd llama.cpp && ./main -m ../llama-2-7b-combined/ggml-model-q6_K.bin --color -p "以下是10条文明乘车的建议：" -n 256

main: build = 916 (b5472ea)
main: seed  = 1690444520
llama.cpp: loading model from ../llama-2-7b-combined/ggml-model-q6_K.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 18 (mostly Q6_K)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 5723.73 MB (+  256.00 MB per state)
llama_